# MusicGen-Large LoRA ファインチューニング (A100推奨)

**目的**: 大規模データセット（約18,000曲）を使い、MusicGen-Large に LoRA を適用して学習します。
Google Colabのディスク容量制限を回避するため、**順次処理戦略（抽出→トークン化→削除）**を採用しています。

---

## 📋 事前準備

1. **Google Colabのランタイム**: GPU (A100推奨) を選択
2. **シークレットの設定**: Colabの「🔑シークレット」メニューから以下を設定
   - `WANDB_API_KEY`: WandB APIキー
   - `HF_TOKEN`: Hugging Face トークン
3. **Google Drive**: データセット（`archive_batch_xxxx.zip` 形式）を配置

---

## 🚀 使い方

1. セルを順番に実行
2. シークレットから自動的にAPIキーを取得
3. WandBで学習進捗をリアルタイム監視: https://wandb.ai/

# セル 1 — 依存関係インストール

In [ ]:
# @title fix environment - must restart notebook after!!
ROOT_DIR = f'/content'

%cd {ROOT_DIR}
!pip uninstall torch torchvision torchaudio -y
!pip install -q timm==1.0.22
!pip install -q fastai==2.8.5
!pip install -q torch==2.9.0
!pip install -q torchaudio==2.9.0
!pip install -q torchtext==0.6.0 --index-url https://download.pytorch.org/whl/cu121
!pip install -q torchvision==0.24.0

!pip install -q triton==3.5.0

import torch
import torchaudio
import torchtext
import torchvision

print(torch.__version__)
print(torchaudio.__version__)
print(torchtext.__version__)
print(torchvision.__version__)

!pip install -q xformers==0.0.29.post1 --index-url https://download.pytorch.org/whl/cu121 # cuda 12.1 version

!run -m xformers.info
!python os.kill(os.getpid(), signal.SIGKILL)

# ^^^ @baltigor

In [ ]:
# @title download and install

import os
ROOT_DIR = '/content'
AUDIOCRAFT_DIR = os.path.join(ROOT_DIR, 'audiocraft')
AUDIOCRAFT_REPO = 'https://github.com/facebookresearch/audiocraft.git'

# audiocraftが既に存在する場合は削除
if os.path.exists(AUDIOCRAFT_DIR):
    import shutil
    shutil.rmtree(AUDIOCRAFT_DIR)

# audiocraftをクローン
!git clone {AUDIOCRAFT_REPO} {AUDIOCRAFT_DIR}

# audiocraftディレクトリに移動してインストール
%cd {AUDIOCRAFT_DIR}
!pip install -q -e .

# その他の依存関係をインストール
!pip install -q dora-search numba wandb loralib

# ^^^ @baltigor

!python -m pip install -U pip setuptools wheel
!python -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

!python -m pip install laion-clap
!python -m pip install -U transformers==4.30.0

# インストール確認
print("✓ インストール完了")
try:
    from audiocraft.models import CompressionModel, MusicGen
    print("✓ audiocraft.models のインポート成功")
except ImportError as e:
    print(f"⚠️ インポートエラー: {e}")
    print("ランタイムを再起動してください: ランタイム > ランタイムを再起動")

# セル 2 — Drive マウントと設定

Google Driveをマウントし、データセットのパスを設定します。

In [ ]:
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive',force_remount=True)

# 設定
# ZIPファイルが格納されているディレクトリ
DRIVE_ARCHIVE_DIR = '/content/drive/MyDrive/Archive_Wavs'

# 一時展開先（Colabローカル）
TEMP_DATA_DIR = '/content/temp_dataset'

# トークン保存先（Google Drive推奨、または容量があればローカル）
# ※再開可能にするためDrive推奨ですが、I/Oが遅い場合はローカルにして最後にバックアップする戦略も可
# ここではDriveに直接保存する設定にします
TOKEN_DIR = '/content/drive/MyDrive/MusicGen_Tokens'

os.makedirs(TOKEN_DIR, exist_ok=True)
os.makedirs(TEMP_DATA_DIR, exist_ok=True)

print(f'ZIP格納場所: {DRIVE_ARCHIVE_DIR}')
print(f'一時展開先: {TEMP_DATA_DIR}')
print(f'トークン保存先: {TOKEN_DIR}')

# ZIPファイルの確認
zip_files = sorted(list(Path(DRIVE_ARCHIVE_DIR).glob('archive_batch_*.zip')))
print(f'検出されたZIPファイル数: {len(zip_files)}')
if len(zip_files) > 0:
    print(f'例: {zip_files[0].name}')
else:
    print('⚠️ ZIPファイルが見つかりません。パスを確認してください。')

# セル 2.5 — WandBとHugging Faceの認証設定

シークレットからAPIキーを取得してログインします。

In [ ]:
from google.colab import userdata
import wandb
import os

# シークレットからAPIキーを取得
try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')

    # WandBログイン
    wandb.login(key=WANDB_API_KEY)
    print("✓ WandBにログインしました")

    # Hugging Face トークンを環境変数に設定
    os.environ['HF_TOKEN'] = HF_TOKEN
    print("✓ Hugging Face トークンを設定しました")

except Exception as e:
    print(f"⚠️ 認証エラー: {e}")
    print("Colabの「シークレット」メニューから以下のキーを設定してください:")
    print("  - WANDB_API_KEY: WandBのAPIキー")
    print("  - HF_TOKEN: Hugging FaceのAPIトークン")

# セル 2.6 — 🧪 少数サンプルでの動作確認テスト

最初のZIPファイルから少数のファイルを抽出して、パイプライン全体の動作確認を行います。

In [ ]:
import time
import json
import os
import shutil
from pathlib import Path
import torch
import torchaudio
from audiocraft.models import CompressionModel, MusicGen
from torch.utils.data import Dataset, DataLoader
import loralib as lora
import torch.nn as nn

# テスト設定
TEST_MODE = True
NUM_TEST_SAMPLES = 5
TEST_EPOCHS = 2

print("=" * 60)
print("🧪 少数サンプルテストモード")
print("=" * 60)

# テスト用のディレクトリ
TEST_DIR = '/content/test_workspace'
TEST_WAV_DIR = os.path.join(TEST_DIR, 'wavs')
TEST_TOKEN_DIR = os.path.join(TEST_DIR, 'tokens')
os.makedirs(TEST_WAV_DIR, exist_ok=True)
os.makedirs(TEST_TOKEN_DIR, exist_ok=True)

# 1. データの抽出（最初のZIPから数ファイルだけ）
print("📁 ステップ1: テストデータの準備")
if len(zip_files) > 0:
    target_zip = zip_files[0]
    print(f"  ZIPファイル: {target_zip.name}")
    # 一時的に展開して数ファイルだけコピー
    !mkdir -p /content/temp_extract
    !tar -xzf "{target_zip}" -C /content/temp_extract

    extracted_files = list(Path('/content/temp_extract').rglob('*.wav'))
    print(f"  抽出されたファイル数: {len(extracted_files)}")

    for i, f in enumerate(extracted_files[:NUM_TEST_SAMPLES]):
        shutil.copy(f, TEST_WAV_DIR)
        print(f"  コピー: {f.name}")

    # 掃除
    !rm -rf /content/temp_extract
else:
    print("⚠️ ZIPファイルがないため、ダミーデータを作成します")
    # ダミーWAV作成（正弦波）
    sr = 32000
    for i in range(NUM_TEST_SAMPLES):
        t = torch.linspace(0, 5, sr * 5)
        wav = torch.sin(2 * 3.14159 * 440 * t).unsqueeze(0)
        torchaudio.save(os.path.join(TEST_WAV_DIR, f'test_{i}.wav'), wav, sr)

test_paths = list(Path(TEST_WAV_DIR).glob('*.wav'))
print(f"✓ {len(test_paths)}個のテストファイルを用意")
print()

# 2. トークナイズのテスト
print("🎵 ステップ2: トークナイズのテスト")
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

for i, p in enumerate(test_paths):
    try:
        wav, sr = torchaudio.load(p)
        if sr != 32000:
            resampler = torchaudio.transforms.Resample(sr, 32000)
            wav = resampler(wav)
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        wav = wav.unsqueeze(0).to('cuda')

        with torch.no_grad():
            encoded_frames = compression_model.encode(wav)
            tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

        outp = os.path.join(TEST_TOKEN_DIR, p.stem + '.pt')
        torch.save({'tokens': tokens.cpu(), 'path': str(p)}, outp)
        print(f"  ✓ トークン化完了: {p.name} -> {tokens.shape}")
    except Exception as e:
        print(f"  ✗ エラー: {e}")

del compression_model
torch.cuda.empty_cache()
print()

# 3. データローダのテスト
print("📦 ステップ3: データローダのテスト")
class TestTokenDataset(Dataset):
    def __init__(self, token_dir, max_length=1500):
        self.files = sorted(list(Path(token_dir).glob('*.pt')))
        self.max_length = max_length
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        tokens = data['tokens']
        if tokens.dim() == 3: tokens = tokens.squeeze(0)
        seq_len = tokens.shape[-1]
        if seq_len > self.max_length:
            tokens = tokens[:, :self.max_length]
        elif seq_len < self.max_length:
            pad = self.max_length - seq_len
            tokens = torch.nn.functional.pad(tokens, (0, pad), value=0)
        return tokens

test_dataset = TestTokenDataset(TEST_TOKEN_DIR)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)
for batch in test_loader:
    print(f"✓ バッチ形状: {batch.shape}")
    break
print()

# 4. モデルとLoRAのテスト
print("🤖 ステップ4: モデルロードとLoRA適用のテスト")
model = MusicGen.get_pretrained('facebook/musicgen-large')
model.to('cuda')
model.eval()

def apply_lora_to_linear(module, r=8, lora_alpha=32, lora_dropout=0.1, parent_name='', count=[0]):
    for name, child in list(module.named_children()):
        if isinstance(child, nn.Linear) and count[0] < 3:  # テスト用制限
            lora_linear = lora.Linear(child.in_features, child.out_features, r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout, bias=(child.bias is not None))
            lora_linear.weight.data = child.weight.data.clone()
            if child.bias is not None: lora_linear.bias.data = child.bias.data.clone()
            setattr(module, name, lora_linear)
            count[0] += 1
            print(f"  ✓ LoRA適用: {parent_name}.{name}")
        else:
            apply_lora_to_linear(child, r, lora_alpha, lora_dropout, f"{parent_name}.{name}" if parent_name else name, count)

apply_lora_to_linear(model.lm, r=8)
print("✓ LoRA適用完了")
print()

# 5. 学習ループのテスト
print("🏋️ ステップ5: 学習ループのテスト")
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
model.train()
for epoch in range(TEST_EPOCHS):
    for batch in test_loader:
        batch = batch.to('cuda')
        out = model.lm.forward(batch)
        loss = criterion(out, batch.float())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"  Epoch {epoch+1}, Loss: {loss.item():.4f}")
        break
print()

# 6. 生成テスト
print("🎶 ステップ6: 生成テスト")
model.eval()
model.set_generation_params(duration=5)
with torch.no_grad():
    wav = model.generate(["Test melody"])
print(f"✓ 生成完了: {wav.shape}")

del model
torch.cuda.empty_cache()
print("✅ 全テスト完了")

# セル 3.5 — トークナイズ速度計測

最初の100ファイルで処理速度を計測し、全データセットの処理時間を推定します。


In [ ]:
import time
import os
import shutil
from pathlib import Path
import torch
import torchaudio
from tqdm import tqdm
from audiocraft.models import CompressionModel

print("⏱️ トークナイズ速度計測")
print("=" * 60)

# モデルロード
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

# テスト用サンプル数
NUM_SAMPLES = 100

# 最初のZIPファイルからサンプルを抽出
if len(zip_files) > 0:
    target_zip = zip_files[0]
    print(f"📦 テストZIP: {target_zip.name}")
    
    # 一時展開
    test_extract_dir = '/content/temp_speed_test'
    if os.path.exists(test_extract_dir):
        shutil.rmtree(test_extract_dir)
    os.makedirs(test_extract_dir, exist_ok=True)
    
    !tar -xzf "{target_zip}" -C "{test_extract_dir}"
    
    wav_files = list(Path(test_extract_dir).rglob('*.wav'))[:NUM_SAMPLES]
    print(f"📊 テストファイル数: {len(wav_files)}")
    
    # 速度計測
    start_time = time.time()
    processed = 0
    
    for wav_path in tqdm(wav_files, desc="計測中"):
        try:
            wav, sr = torchaudio.load(str(wav_path))
            if sr != 32000:
                resampler = torchaudio.transforms.Resample(sr, 32000)
                wav = resampler(wav)
            if wav.shape[0] > 1:
                wav = wav.mean(dim=0, keepdim=True)
            wav = wav.unsqueeze(0).to('cuda')
            
            with torch.no_grad():
                encoded_frames = compression_model.encode(wav)
                tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)
            
            processed += 1
        except Exception as e:
            print(f"エラー: {wav_path.name} - {e}")
            continue
    
    elapsed = time.time() - start_time
    files_per_sec = processed / elapsed if elapsed > 0 else 0
    
    # 全ファイル数の推定
    total_zips = len(zip_files)
    estimated_files_per_zip = len(list(Path(test_extract_dir).rglob('*.wav')))
    total_estimated_files = total_zips * estimated_files_per_zip
    
    # 推定時間計算
    estimated_total_time = (total_estimated_files / files_per_sec) / 3600  # 時間単位
    
    print("\n" + "=" * 60)
    print("📈 計測結果")
    print("=" * 60)
    print(f"処理速度: {files_per_sec:.2f} ファイル/秒")
    print(f"推定総ファイル数: {total_estimated_files:,} ファイル")
    print(f"推定総処理時間: {estimated_total_time:.2f} 時間")
    print(f"ZIPファイル数: {total_zips}")
    print("=" * 60)
    
    # クリーンアップ
    shutil.rmtree(test_extract_dir)
else:
    print("⚠️ ZIPファイルが見つかりません")

del compression_model
torch.cuda.empty_cache()
print("\n✅ 速度計測完了")


# セル 3 — 順次処理によるトークナイズ (Sequential Tokenization)

**戦略**: ZIPファイルを1つずつ展開 → トークン化 → WAV削除 を繰り返します。
これにより、Colabのディスク容量を圧迫せずに大量のデータを処理できます。

In [ ]:
import time
import shutil
import glob
import os
from pathlib import Path
import torch
import torchaudio
from tqdm import tqdm
from audiocraft.models import CompressionModel
import wandb

# WandB初期化
try:
    from google.colab import userdata
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    if not wandb.api.api_key: wandb.login(key=WANDB_API_KEY)
    wandb.init(project='musicgen-lora-finetune', name='sequential_tokenization', job_type='preprocessing')
    use_wandb = True
except: use_wandb = False

# モデルロード
print('EnCodecモデルをロード中...')
compression_model = CompressionModel.get_pretrained('facebook/encodec_32khz')
compression_model.to('cuda')
compression_model.eval()

# 処理済みファイルの確認
existing_tokens = set([Path(f).stem for f in glob.glob(os.path.join(TOKEN_DIR, '*.pt'))])
print(f'既にトークン化済み: {len(existing_tokens)} ファイル')

total_processed = 0
start_time = time.time()

# ZIPファイルごとのループ
for zip_idx, zip_file in enumerate(tqdm(zip_files, desc='ZIPバッチ処理')):
    print(f'\n📦 処理中: {zip_file.name} ({zip_idx+1}/{len(zip_files)})')

    # 1. 展開 (Extract)
    if os.path.exists(TEMP_DATA_DIR):
        shutil.rmtree(TEMP_DATA_DIR)
    os.makedirs(TEMP_DATA_DIR, exist_ok=True)

    print('  ↳ 展開中...')
    !tar -xzf "{zip_file}" -C "{TEMP_DATA_DIR}"

    wav_files = list(Path(TEMP_DATA_DIR).rglob('*.wav'))
    print(f'  ↳ {len(wav_files)} 個のWAVファイルを検出')

    # 2. トークン化 (Tokenize)
    batch_processed = 0
    for wav_path in tqdm(wav_files, desc='  ↳ トークン化', leave=False):
        if wav_path.stem in existing_tokens:
            continue

        try:
            wav, sr = torchaudio.load(str(wav_path))
            if sr != 32000:
                resampler = torchaudio.transforms.Resample(sr, 32000)
                wav = resampler(wav)
            if wav.shape[0] > 1:
                wav = wav.mean(dim=0, keepdim=True)
            wav = wav.unsqueeze(0).to('cuda')

            with torch.no_grad():
                encoded_frames = compression_model.encode(wav)
                tokens = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

            outp = os.path.join(TOKEN_DIR, wav_path.stem + '.pt')
            torch.save({'tokens': tokens.cpu(), 'path': str(wav_path)}, outp)

            batch_processed += 1
            total_processed += 1

        except Exception as e:
            print(f'Error: {wav_path.name} - {e}')
            continue

    # 3. クリーンアップ (Cleanup)
    print('  ↳ クリーンアップ中...')
    shutil.rmtree(TEMP_DATA_DIR)

    # ログ
    if use_wandb:
        wandb.log({
            'processed_files': total_processed,
            'processed_zips': zip_idx + 1,
            'files_in_batch': batch_processed
        })

print(f'\n✅ 全処理完了: {total_processed} ファイルを新規にトークン化しました')
if use_wandb: wandb.finish()

del compression_model
torch.cuda.empty_cache()

# セル 4 — DeepSpeed 設定ファイル作成

In [ ]:
import json

ds_config = {
  "train_batch_size": 16,
  "train_micro_batch_size_per_gpu": 2,
  "gradient_accumulation_steps": 8,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 1e-4,
      "weight_decay": 0.01
    }
  },
  "fp16": {
    "enabled": True
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": True
    },
    "offload_param": {
      "device": "cpu"
    },
    "overlap_comm": True,
    "contiguous_gradients": True
  }
}

with open('ds_config.json','w') as f:
    json.dump(ds_config, f, indent=2)

print('ds_config.json を作成しました')

# セル 5 — モデル準備とLoRA適用

In [ ]:
import loralib as lora
import torch.nn as nn
import torch
from audiocraft.models import MusicGen

# モデルロード
print('MusicGen-Large をロード中...')
model = MusicGen.get_pretrained('facebook/musicgen-large')
model.eval()

# LoRA適用関数
def apply_lora_to_linear(module, r=8, lora_alpha=32, lora_dropout=0.1, parent_name=''):
    for name, child in list(module.named_children()):
        full_name = f"{parent_name}.{name}" if parent_name else name
        if isinstance(child, nn.Linear):
            lora_linear = lora.Linear(
                child.in_features, child.out_features, r=r,
                lora_alpha=lora_alpha, lora_dropout=lora_dropout, bias=(child.bias is not None)
            )
            lora_linear.weight.data = child.weight.data.clone()
            if child.bias is not None:
                lora_linear.bias.data = child.bias.data.clone()
            setattr(module, name, lora_linear)
            print(f'Applied LoRA to: {full_name}')
        else:
            apply_lora_to_linear(child, r, lora_alpha, lora_dropout, full_name)

print('\nLoRAを適用中...')
apply_lora_to_linear(model.lm, r=8)

# 勾配設定
for p in model.parameters(): p.requires_grad = False
for name, p in model.named_parameters():
    if 'lora_' in name: p.requires_grad = True

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f'\n学習可能パラメータ: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)')

# セル 6 — データローダ作成

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from pathlib import Path
import random

class TokenDataset(Dataset):
    def __init__(self, token_dir, max_length=1500):
        self.files = sorted(list(Path(token_dir).glob('*.pt')))
        self.max_length = max_length
        print(f'Dataset initialized with {len(self.files)} samples')

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        tokens = data['tokens']
        if tokens.dim() == 3: tokens = tokens.squeeze(0)

        seq_len = tokens.shape[-1]
        if seq_len > self.max_length:
            start = random.randint(0, seq_len - self.max_length)
            tokens = tokens[:, start:start + self.max_length]
        elif seq_len < self.max_length:
            pad = self.max_length - seq_len
            tokens = torch.nn.functional.pad(tokens, (0, pad), value=0)
        return tokens

dataset = TokenDataset(TOKEN_DIR, max_length=1500)
train_size = int(0.95 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, num_workers=2, pin_memory=True)

print(f'Train: {len(train_dataset)}, Valid: {len(valid_dataset)}')

In [ ]:
# 最終モデル保存（学習完了後）
print('💾 最終モデルを保存中...')
final_checkpoint = {
    'epoch': num_epochs - 1,
    'global_step': global_step,
    'model_state': model.state_dict(),
    'opt_state': optimizer.state_dict(),
    'loss': avg_loss,
    'timestamp': time.strftime("%Y%m%d_%H%M%S")
}
final_lora_checkpoint = {
    'epoch': num_epochs - 1,
    'global_step': global_step,
    'lora_weights': extract_lora_weights(model),
    'loss': avg_loss,
    'timestamp': final_checkpoint['timestamp']
}

torch.save(final_checkpoint, os.path.join(FULL_MODEL_DIR, 'final_model.pt'))
torch.save(final_lora_checkpoint, os.path.join(LORA_DIR, 'final_lora.pt'))
print('✓ 最終モデルを保存しました')

# WandBアーティファクトとして保存
if use_wandb:
    try:
        if os.path.exists(os.path.join(LORA_DIR, 'best_lora.pt')):
            artifact = wandb.Artifact('best_lora_weights', type='model')
            artifact.add_file(os.path.join(LORA_DIR, 'best_lora.pt'))
            wandb.log_artifact(artifact)
            print('✓ ベストLoRA重みをWandBに保存しました')
        
        artifact_final = wandb.Artifact('final_lora_weights', type='model')
        artifact_final.add_file(os.path.join(LORA_DIR, 'final_lora.pt'))
        wandb.log_artifact(artifact_final)
        print('✓ 最終LoRA重みをWandBに保存しました')
        
        wandb.finish()
        print('✓ WandBセッションを終了しました')
    except Exception as e:
        print(f'⚠️ WandBアーティファクト保存エラー: {e}')

print('✅ 全処理完了')


# セル 7 — 学習ループ (WandB + Checkpoints)

In [ ]:
import os
import time
import torch
import torch.nn as nn
import wandb
from tqdm import tqdm
from google.colab import userdata

# WandB初期化
try:
    WANDB_API_KEY = userdata.get('WANDB_API_KEY')
    if not wandb.api.api_key: wandb.login(key=WANDB_API_KEY)
    wandb.init(project='musicgen-lora-finetune', name='training_run', tags=['musicgen', 'lora', 'a100'])
    use_wandb = True
except: use_wandb = False

device = torch.device('cuda')
model = model.to(device)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=0.01)

# 保存先設定
CKPT_DIR = '/content/drive/MyDrive/MusicGen_Checkpoints'
LORA_DIR = os.path.join(CKPT_DIR, 'lora_weights')
FULL_MODEL_DIR = os.path.join(CKPT_DIR, 'full_models')
os.makedirs(LORA_DIR, exist_ok=True)
os.makedirs(FULL_MODEL_DIR, exist_ok=True)

def extract_lora_weights(model):
    return {name: param.cpu().detach().clone() for name, param in model.named_parameters() if 'lora_' in name and param.requires_grad}

def save_checkpoint(model, optimizer, epoch, global_step, loss, checkpoint_type='regular'):
    checkpoint = {
        'epoch': epoch, 'global_step': global_step, 'model_state': model.state_dict(),
        'opt_state': optimizer.state_dict(), 'loss': loss, 'best_loss': best_loss,
        'timestamp': time.strftime("%Y%m%d_%H%M%S")
    }
    lora_checkpoint = {
        'epoch': epoch, 'global_step': global_step, 'lora_weights': extract_lora_weights(model),
        'loss': loss, 'timestamp': checkpoint['timestamp']
    }

    if checkpoint_type == 'regular':
        torch.save(checkpoint, os.path.join(CKPT_DIR, 'latest.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, 'latest_lora.pt'))
    elif checkpoint_type == 'best':
        torch.save(checkpoint, os.path.join(FULL_MODEL_DIR, 'best_model.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, 'best_lora.pt'))
        print(f'  🏆 ベストモデル保存: Loss {loss:.4f}')
    elif checkpoint_type == 'epoch':
        torch.save(checkpoint, os.path.join(FULL_MODEL_DIR, f'checkpoint_epoch_{epoch+1}.pt'))
        torch.save(lora_checkpoint, os.path.join(LORA_DIR, f'lora_epoch_{epoch+1}.pt'))
    elif checkpoint_type == 'step':
        torch.save(checkpoint, os.path.join(CKPT_DIR, f'checkpoint_step_{global_step}.pt'))

# 再開処理
start_epoch = 0
global_step = 0
best_loss = float('inf')
latest_ckpt = os.path.join(CKPT_DIR, 'latest.pt')
if os.path.exists(latest_ckpt):
    print(f'🔄 再開: {latest_ckpt}')
    ckpt = torch.load(latest_ckpt, map_location='cpu')
    model.load_state_dict(ckpt['model_state'], strict=False)
    optimizer.load_state_dict(ckpt['opt_state'])
    start_epoch = ckpt['epoch'] + 1
    global_step = ckpt.get('global_step', 0)
    best_loss = ckpt.get('best_loss', float('inf'))

criterion = nn.MSELoss()
num_epochs = 5
accum_steps = 8

print('\n🚀 学習開始')
for epoch in range(start_epoch, num_epochs):
    model.train()
    epoch_loss = 0.0
    optimizer.zero_grad()

    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch_idx, tokens in enumerate(pbar):
        try:
            tokens = tokens.to(device)
            outputs = model.lm.forward(tokens)
            loss = criterion(outputs, tokens.float()) / accum_steps
            loss.backward()

            if (batch_idx + 1) % accum_steps == 0:
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), 1.0)
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

                if use_wandb and global_step % 10 == 0:
                    wandb.log({'train/loss': loss.item() * accum_steps, 'train/step': global_step})
                if global_step % 500 == 0:
                    save_checkpoint(model, optimizer, epoch, global_step, loss.item() * accum_steps, 'step')

            epoch_loss += loss.item() * accum_steps
            pbar.set_postfix({'loss': f'{loss.item() * accum_steps:.4f}'})
        except Exception as e:
            print(f'Error: {e}')
            continue

    avg_loss = epoch_loss / len(train_loader)
    print(f'\nEpoch {epoch+1} Loss: {avg_loss:.4f}')

    if use_wandb: wandb.log({'epoch/loss': avg_loss, 'epoch': epoch+1})

    if avg_loss < best_loss:
        best_loss = avg_loss
        save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'best')

    save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'regular')
    save_checkpoint(model, optimizer, epoch, global_step, avg_loss, 'epoch')

print('🎉 学習完了')

# セル 8 — 楽曲生成

In [ ]:
from audiocraft.data.audio import audio_write
from IPython.display import Audio, display

print('🎵 楽曲生成')
model.eval()
model.set_generation_params(use_sampling=True, top_k=250, duration=30)

prompts = [
    "A dynamic heavy metal song with fast drums and guitar solo",
    "Relaxing jazz piano with soft background ambience",
    "Upbeat electronic dance music with strong bass"
]

with torch.no_grad():
    wav = model.generate(prompts)

output_dir = '/content/drive/MyDrive/MusicGen_Generated'
os.makedirs(output_dir, exist_ok=True)

for idx, one_wav in enumerate(wav):
    filename = os.path.join(output_dir, f"generated_{idx}")
    audio_write(filename, one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)
    print(f'\nPrompt: {prompts[idx]}')
    display(Audio(filename + ".wav"))